In [1]:
import json
import os
import copy
import astroddpm

In [2]:
CONFIG_FOLDER = os.path.join(os.path.dirname(astroddpm.__file__),'config')
CONFIG_FOLDER

'/mnt/home/dheurtel/astroddpm/astroddpm/config'

In [5]:
ALL_MODELS = os.path.join(CONFIG_FOLDER,'MODELS.json')
with open(ALL_MODELS) as f:
    ALL_MODELS_DICT = json.load(f)
print(ALL_MODELS_DICT.keys())

dict_keys(['DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_1000_bottleneck_32', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_1000_bottleneck_16', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_8000_bottleneck_16', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_4000_bottleneck_32', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_8000_bottleneck_32', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_4000_bottleneck_16', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_8000_bottleneck_32', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_8000_bottleneck_16', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_betamin_0.01_step', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_betamin_0.05_invsqrt', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_betamin_0.05_step', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_beta

In [6]:
config_init = ALL_MODELS_DICT['DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_20_invsqrt']

config_init

{'diffusion_model': {'sde': {'type': 'DiscreteVPSDE',
   'beta_min': 0.1,
   'beta_max': 20.0,
   'N': 1000},
  'network': {'in_c': 1,
   'out_c': 1,
   'first_c': 20,
   'sizes': [256, 128, 64, 32],
   'num_blocks': 1,
   'n_steps': 1000,
   'time_emb_dim': 100,
   'dropout': 0,
   'attention': [],
   'normalisation': 'GN',
   'padding_mode': 'circular',
   'eps_norm': 1e-05,
   'skiprescale': True,
   'type': 'ResUNet',
   'discretization': 'discrete',
   'embedding_mode': None},
  'type': 'DiscreteSBM',
  'ps': {'type': 'cmb_h_ombh2'}},
 'dataloaders': {'train_batch_size': 64,
  'test_batch_size': 128,
  'num_workers': 8,
  'seed': 33,
  'split': 0.1,
  'dataset': {'dir': '/mnt/home/dheurtel/ceph/01_data/CATS_MHD_BPROJ_DENSITY/',
   'type': 'npy files',
   'transforms': ['Random_Rotate90'],
   'name': 'CATS_MHD_BPROJ_DENSITY'},
  'name': 'CATS_MHD_BPROJ_DENSITY'},
 'optimizer': {'type': 'AdamW', 'lr': 0.001, 'weight_decay': 0.0001},
 'scheduler': {'type': 'invsqrt',
  'warmup': 100,

EXP1 is false as is

## EXP1

In [20]:
sizes_list = [[256, 128, 64, 32], [256, 128, 64, 32, 16,], [256, 128, 64, 32, 16, 8,]]
N_list = [1000, 4000, 8000]
sde_list = ['DiscreteVPSDE', 'DiscreteSigmaVPSDE']

In [21]:
power_spectrum_name = 'cmb_256_8arcmippixel'

In [22]:
all_configs = {}
for sde in sde_list:
    for n in N_list:
        for sizes in sizes_list:
            config = copy.deepcopy(config_init)
            config['diffusion_model']['sde']['type'] = sde
            config['diffusion_model']['sde']['N'] = n
            config['diffusion_model']['network']['n_steps'] = n
            if sde == 'DiscreteSigmaVPSDE':
                config['diffusion_model']['sde']['power_spectrum_name'] = power_spectrum_name
            config['diffusion_model']['network']['sizes'] = sizes
            if sde == 'DiscreteVPSDE':
                config['model_id'] = 'DiscreteSBM_VPSDE_MHD_BPROJ_N_{}_bottleneck_{}'.format(n,sizes[-1])
            else:
                config['model_id'] = 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_{}_bottleneck_{}'.format(n,sizes[-1])
            all_configs[config['model_id']] = config
            

In [23]:
with open(os.path.join(CONFIG_FOLDER,'exp1','config.json'), 'w') as f:
    json.dump(all_configs, f, indent=4)

In [24]:
for key in all_configs.keys():
    print(key, all_configs[key]['diffusion_model']['sde']['N'], all_configs[key]['diffusion_model']['sde']['type'], all_configs[key]['diffusion_model']['network']['n_steps'], all_configs[key]['diffusion_model']['network']['sizes'][-1])

DiscreteSBM_VPSDE_MHD_BPROJ_N_1000_bottleneck_32 1000 DiscreteVPSDE 1000 32
DiscreteSBM_VPSDE_MHD_BPROJ_N_1000_bottleneck_16 1000 DiscreteVPSDE 1000 16
DiscreteSBM_VPSDE_MHD_BPROJ_N_1000_bottleneck_8 1000 DiscreteVPSDE 1000 8
DiscreteSBM_VPSDE_MHD_BPROJ_N_4000_bottleneck_32 4000 DiscreteVPSDE 4000 32
DiscreteSBM_VPSDE_MHD_BPROJ_N_4000_bottleneck_16 4000 DiscreteVPSDE 4000 16
DiscreteSBM_VPSDE_MHD_BPROJ_N_4000_bottleneck_8 4000 DiscreteVPSDE 4000 8
DiscreteSBM_VPSDE_MHD_BPROJ_N_8000_bottleneck_32 8000 DiscreteVPSDE 8000 32
DiscreteSBM_VPSDE_MHD_BPROJ_N_8000_bottleneck_16 8000 DiscreteVPSDE 8000 16
DiscreteSBM_VPSDE_MHD_BPROJ_N_8000_bottleneck_8 8000 DiscreteVPSDE 8000 8
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32 1000 DiscreteSigmaVPSDE 1000 32
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16 1000 DiscreteSigmaVPSDE 1000 16
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_8 1000 DiscreteSigmaVPSDE 1000 8
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32 4000 Discret

In [19]:
## Adding the content of config.json to MODELS.json because I forgot to do it before
with open(os.path.join(CONFIG_FOLDER,'exp1','config.json')) as f:
    all_configs = json.load(f)

with open(ALL_MODELS) as f:
    ALL_MODELS_DICT = json.load(f)

ALL_MODELS_DICT.update(all_configs)

with open(ALL_MODELS, 'w') as f:
    json.dump(ALL_MODELS_DICT, f, indent=4)

### Summary:

In [ ]:
## Adding the content of config.json to MODELS.json because I forgot to do it before
with open(os.path.join(CONFIG_FOLDER,'exp1','config.json')) as f:
    all_configs = json.load(f)

with open(ALL_MODELS) as f:
    ALL_MODELS_DICT = json.load(f)

ALL_MODELS_DICT.update(all_configs)

with open(ALL_MODELS, 'w') as f:
    json.dump(ALL_MODELS_DICT, f, indent=4)

## EXP2

In [16]:
sizes_list = [[256, 128, 64, 32], [256, 128, 64, 32, 16,]]
N_list = [1000, 4000]
sde_list = ['DiscreteSigmaVPSDE']
first_c_list = [4, 6, 10]

In [17]:
power_spectrum_name = 'cmb_256_8arcmippixel'

In [18]:
all_configs = {}
for sde in sde_list:
    for n in N_list:
        for sizes in sizes_list:
            for first_c in first_c_list:
                config = copy.deepcopy(config_init)
                config['diffusion_model']['sde']['type'] = sde
                config['diffusion_model']['sde']['N'] = n
                config['diffusion_model']['network']['n_steps'] = n
                config['diffusion_model']['network']['first_c'] = first_c
                config['diffusion_model']['sde']['power_spectrum_name'] = power_spectrum_name
                config['diffusion_model']['network']['sizes'] = sizes
                config['epochs'] = 20000
                config['model_id'] = 'DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_{}_bottleneck_{}_firstc_{}'.format(n,sizes[-1], first_c)
                all_configs[config['model_id']] = config

In [21]:
with open(os.path.join(CONFIG_FOLDER,'exp2','config.json'), 'w') as f:
    json.dump(all_configs, f, indent=4)

In [22]:
for key in all_configs.keys():
    print(key)

DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_4
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_6
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_10
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_4
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_6
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_10
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_firstc_4
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_firstc_6
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_firstc_10
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16_firstc_4
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16_firstc_6
DiscreteSBM_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16_firstc_10


## EXP3 NON DDPM MATH Training

In [4]:
sizes_list = [[256, 128, 64, 32], [256, 128, 64, 32, 16,]]
N_list = [1000, 4000, 8000]
sde_list = ['DiscreteSigmaVPSDE', 'DiscreteVPSDE']
power_spectrum_name = 'cmb_256_8arcmippixel'

In [7]:
all_configs = {}
for sde in sde_list:
    for n in N_list:
        for sizes in sizes_list:
            config = copy.deepcopy(config_init)
            config['diffusion_model']['sde']['type'] = sde
            config['diffusion_model']['sde']['N'] = n
            config['diffusion_model']['network']['n_steps'] = n
            if sde == 'DiscreteSigmaVPSDE':
                config['diffusion_model']['sde']['power_spectrum_name'] = power_spectrum_name
            config['diffusion_model']['sde']['ddpm_math'] = False
            config['epochs'] = 20000
            config['diffusion_model']['network']['sizes'] = sizes
            if sde == 'DiscreteVPSDE':
                config['model_id'] = 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_{}_bottleneck_{}'.format(n,sizes[-1])
            else:
                config['model_id'] = 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_{}_bottleneck_{}'.format(n,sizes[-1])
            all_configs[config['model_id']] = config

In [8]:
print(all_configs.keys())

dict_keys(['DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_8000_bottleneck_32', 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_8000_bottleneck_16', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_1000_bottleneck_32', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_1000_bottleneck_16', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_4000_bottleneck_32', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_4000_bottleneck_16', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_8000_bottleneck_32', 'DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_8000_bottleneck_16'])


In [9]:
with open(os.path.join(CONFIG_FOLDER,'exp3','config.json'), 'w') as f:
    json.dump(all_configs, f, indent=4)

In [10]:
str_keys = ""
for key in all_configs.keys():
    print(key)
    str_keys += key + " "

DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_8000_bottleneck_32
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_8000_bottleneck_16
DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_1000_bottleneck_32
DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_1000_bottleneck_16
DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_4000_bottleneck_32
DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_4000_bottleneck_16
DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_8000_bottleneck_32
DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_8000_bottleneck_16


In [11]:
str_keys

'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32 DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16 DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32 DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16 DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_8000_bottleneck_32 DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_8000_bottleneck_16 DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_1000_bottleneck_32 DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_1000_bottleneck_16 DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_4000_bottleneck_32 DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_4000_bottleneck_16 DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_8000_bottleneck_32 DiscreteSBM_ND_VPSDE_MHD_BPROJ_N_8000_bottleneck_16 '

## EXP4

In [17]:
sizes_list = [[256, 128, 64, 32], [256, 128, 64, 32, 16,]]
N_list = [1000, 4000]
sde_list = ['DiscreteSigmaVPSDE']
beta_min_list = [0.01, 0.05]
scheduler_list = ['invsqrt', 'step']
power_spectrum_name = 'cmb_256_8arcmippixel'

In [21]:
all_configs = {}
for sde in sde_list:
    for n in N_list:
        for sizes in sizes_list:
            for beta in beta_min_list:
                for scheduler in scheduler_list:
                    config = copy.deepcopy(config_init)
                    config['diffusion_model']['sde']['type'] = sde
                    config['diffusion_model']['sde']['N'] = n
                    config['diffusion_model']['sde']['beta_min'] = beta
                    config['diffusion_model']['network']['n_steps'] = n
                    config['diffusion_model']['sde']['power_spectrum_name'] = power_spectrum_name
                    config['diffusion_model']['sde']['ddpm_math'] = False
                    config['epochs'] = 20000
                    config['diffusion_model']['network']['sizes'] = sizes
                    if scheduler == 'step':
                        config['scheduler'] = {'type': 'step', 'warmup' : 100, 'max_lr': 2e-3, 'step_period' : 400}
                    config['model_id'] = 'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_{}_bottleneck_{}_betamin_{}_{}'.format(n,sizes[-1], beta, scheduler)
                    all_configs[config['model_id']] = config

In [22]:
with open(os.path.join(CONFIG_FOLDER,'exp4','config.json'), 'w') as f:
    json.dump(all_configs, f, indent=4)

In [23]:
str_keys = ""
for key in all_configs.keys():
    print(key)
    str_keys += key + " "

DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_betamin_0.01_invsqrt
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_betamin_0.01_step
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_betamin_0.05_invsqrt
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_betamin_0.05_step
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_betamin_0.01_invsqrt
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_betamin_0.01_step
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_betamin_0.05_invsqrt
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_betamin_0.05_step
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_betamin_0.01_invsqrt
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_betamin_0.01_step
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_betamin_0.05_invsqrt
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_betamin_0.05_step
DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16_betamin_0.01_invsqrt
Disc

In [24]:
str_keys

'DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_betamin_0.01_invsqrt DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_betamin_0.01_step DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_betamin_0.05_invsqrt DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_betamin_0.05_step DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_betamin_0.01_invsqrt DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_betamin_0.01_step DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_betamin_0.05_invsqrt DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_betamin_0.05_step DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_betamin_0.01_invsqrt DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_betamin_0.01_step DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_betamin_0.05_invsqrt DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_32_betamin_0.05_step DiscreteSBM_ND_SigmaVPSDE_MHD_BPROJ_N_4000_bottleneck_16_betamin_0.01_invsqrt Dis

### EXP5

Custom

### EXP6

In [3]:
sizes_list = [[256, 128, 64, 32], [256, 128, 64, 32, 16,]]
first_c_list = [6, 10, 20]
scheduler_list = ['invsqrt', 'step']

In [4]:
with open(os.path.join(CONFIG_FOLDER, 'exp5', 'config.json'), 'r') as f:
    only_configs = json.load(f)

In [5]:
key = next(iter(only_configs.keys()))
config_init = only_configs[key]

In [6]:
config_init

{'diffusion_model': {'sde': {'type': 'discretevpsde',
   'beta_min': 0.1,
   'beta_max': 20.0,
   'N': 1000},
  'network': {'in_c': 1,
   'out_c': 1,
   'first_c': 20,
   'sizes': [256, 128, 64, 32, 16],
   'num_blocks': 1,
   'n_steps': 4000,
   'time_emb_dim': 100,
   'dropout': 0,
   'attention': [],
   'normalisation': 'GN',
   'padding_mode': 'circular',
   'eps_norm': 1e-05,
   'skiprescale': True,
   'type': 'ResUNet',
   'discretization': 'discrete',
   'embedding_mode': None},
  'type': 'DiscreteSBM',
  'ps': {'type': 'cmb_h_ombh2'}},
 'dataloaders': {'train_batch_size': 64,
  'test_batch_size': 128,
  'num_workers': 8,
  'seed': 33,
  'split': 0.1,
  'dataset': {'dir': '/mnt/home/dheurtel/ceph/01_data/CATS_MHD_BPROJ_DENSITY/',
   'type': 'npy files',
   'transforms': ['Random_Rotate90'],
   'name': 'CATS_MHD_BPROJ_DENSITY'},
  'name': 'CATS_MHD_BPROJ_DENSITY'},
 'optimizer': {'type': 'AdamW', 'lr': 0.001, 'weight_decay': 0.0001},
 'scheduler': {'type': 'invsqrt',
  'warmup': 

In [7]:
all_configs = {}
for sizes in sizes_list:
    for first_c in first_c_list:
        for scheduler in scheduler_list:
            config = copy.deepcopy(config_init)
            config['diffusion_model']['network']['sizes'] = sizes
            if scheduler == 'step':
                config['scheduler'] = {'type': 'step', 'warmup' : 100, 'max_lr': 2e-3, 'step_period' : 400}
            config['model_id'] = 'DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_{}_firstc_{}_{}'.format(sizes[-1], first_c, scheduler)
            all_configs[config['model_id']] = config

In [8]:
str_keys = ""
for key in all_configs.keys():
    print(key)
    str_keys += key + " "

DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_6_invsqrt
DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_6_step
DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_10_invsqrt
DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_10_step
DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_20_invsqrt
DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_20_step
DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_6_invsqrt
DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_6_step
DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_10_invsqrt
DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_10_step
DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_20_invsqrt
DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_20_step


In [9]:
with open(os.path.join(CONFIG_FOLDER,'exp6','config.json'), 'w') as f:
    json.dump(all_configs, f, indent=4)

In [10]:
str_keys

'DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_6_invsqrt DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_6_step DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_10_invsqrt DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_10_step DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_20_invsqrt DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_32_firstc_20_step DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_6_invsqrt DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_6_step DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_10_invsqrt DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_10_step DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_20_invsqrt DiscreteSBM_MultiSigmaVPSDE_MHD_BPROJ_N_1000_bottleneck_16_firstc_20_step '

## EXP 7

In [14]:
sizes_list = [[256, 128, 64, 32], [256, 128, 64, 32, 16,]]
first_c_list = [6, 10, 20]

dataset_name = 'CATS_I'
epochs = 15000

all_configs = {}

In [18]:
for sizes in sizes_list:
    for first_c in first_c_list:
        config = copy.deepcopy(config_init)
        config['diffusion_model']['network']['sizes'] = sizes
        config['diffusion_model']['network']['first_c'] = first_c
        config['model_id'] = 'DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_{}_firstc_{}'.format(sizes[-1], first_c)
        config['dataloaders']['name'] = dataset_name
        config['dataloaders']['dataset']['name'] = dataset_name
        config['dataloaders']['dataset']['dir'] = '/mnt/home/dheurtel/ceph/01_data/CATS_I/'
        config['epochs'] = epochs
        all_configs[config['model_id']] = config

In [19]:
all_configs.keys()

dict_keys(['DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_6', 'DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_10', 'DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_20', 'DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_6', 'DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_10', 'DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_20'])

In [21]:
str_keys = ""
for key in all_configs.keys():
    print(key)
    str_keys += key + " "

DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_6
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_10
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_20
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_6
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_10
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_20


In [24]:
str_keys

'DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_6 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_10 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_20 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_6 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_10 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_20 '

In [23]:
with open(os.path.join(CONFIG_FOLDER,'exp7','config.json'), 'w') as f:
    json.dump(all_configs, f, indent=4)

## EXP 8

In [7]:
sizes_list = [[256, 128, 64, 32], [256, 128, 64, 32, 16,]]
first_c_list = [6, 10]
N_list = [2000, 4000, 8000]
dataset_name = 'CATS_I'
epochs = 15000

all_configs = {}

In [8]:
for sizes in sizes_list:
    for first_c in first_c_list:
        for N in N_list:
            config = copy.deepcopy(config_init)
            config['diffusion_model']['network']['sizes'] = sizes
            config['diffusion_model']['network']['first_c'] = first_c
            config['diffusion_model']['sde']['N'] = N
            config['diffusion_model']['network']['n_steps'] = N
            config['dataloaders']['name'] = dataset_name
            config['dataloaders']['dataset']['name'] = dataset_name
            config['dataloaders']['dataset']['dir'] = '/mnt/home/dheurtel/ceph/01_data/CATS_I/'
            config['epochs'] = epochs
            config['model_id'] = 'DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_{}_bottleneck_{}_firstc_{}'.format(N,sizes[-1], first_c)
            
            all_configs[config['model_id']] = config

In [9]:
str_keys = ""
for key in all_configs.keys():
    print(key)
    str_keys += key + " "

DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_2000_bottleneck_32_firstc_6
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_4000_bottleneck_32_firstc_6
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_8000_bottleneck_32_firstc_6
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_2000_bottleneck_32_firstc_10
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_4000_bottleneck_32_firstc_10
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_8000_bottleneck_32_firstc_10
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_2000_bottleneck_16_firstc_6
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_4000_bottleneck_16_firstc_6
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_8000_bottleneck_16_firstc_6
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_2000_bottleneck_16_firstc_10
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_4000_bottleneck_16_firstc_10
DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_8000_bottleneck_16_firstc_10


In [10]:
str_keys

'DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_2000_bottleneck_32_firstc_6 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_4000_bottleneck_32_firstc_6 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_8000_bottleneck_32_firstc_6 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_2000_bottleneck_32_firstc_10 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_4000_bottleneck_32_firstc_10 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_8000_bottleneck_32_firstc_10 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_2000_bottleneck_16_firstc_6 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_4000_bottleneck_16_firstc_6 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_8000_bottleneck_16_firstc_6 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_2000_bottleneck_16_firstc_10 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_4000_bottleneck_16_firstc_10 DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_8000_bottleneck_16_firstc_10 '

In [11]:
with open(os.path.join(CONFIG_FOLDER,'exp8','config.json'), 'w') as f:
    json.dump(all_configs, f, indent=4)

## EXP 9

In [12]:
ps_config = {"type": "constant",
            "ps_path": "/mnt/home/dheurtel/astroddpm/astroddpm/diffusion/power_spectra/power_spectra/cmb_256_8arcmippixel.npy"}

In [16]:
sizes_list = [[256, 128, 64, 32], [256, 128, 64, 32, 16,]]
first_c_list = [6, 10]
N_list = [1000,2000, 4000]
dataset_name = 'CATS_I'
epochs = 15000

all_configs = {}

In [17]:
for sizes in sizes_list:
    for first_c in first_c_list:
        for N in N_list:
            config = copy.deepcopy(config_init)
            config['diffusion_model']['network']['sizes'] = sizes
            config['diffusion_model']['network']['first_c'] = first_c
            config['diffusion_model']['sde']['N'] = N
            config['diffusion_model']['network']['n_steps'] = N
            config['diffusion_model']['ps'] = ps_config
            config['dataloaders']['name'] = dataset_name
            config['dataloaders']['dataset']['name'] = dataset_name
            config['dataloaders']['dataset']['dir'] = '/mnt/home/dheurtel/ceph/01_data/CATS_I/'
            config['epochs'] = epochs
            config['model_id'] = 'DiscreteSBM_SigmaVPSDE_I_BPROJ_N_{}_bottleneck_{}_firstc_{}'.format(N,sizes[-1], first_c)
            
            all_configs[config['model_id']] = config

In [18]:
str_keys = ""
for key in all_configs.keys():
    print(key)
    str_keys += key + " "

DiscreteSBM_SigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_6
DiscreteSBM_SigmaVPSDE_I_BPROJ_N_2000_bottleneck_32_firstc_6
DiscreteSBM_SigmaVPSDE_I_BPROJ_N_4000_bottleneck_32_firstc_6
DiscreteSBM_SigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_10
DiscreteSBM_SigmaVPSDE_I_BPROJ_N_2000_bottleneck_32_firstc_10
DiscreteSBM_SigmaVPSDE_I_BPROJ_N_4000_bottleneck_32_firstc_10
DiscreteSBM_SigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_6
DiscreteSBM_SigmaVPSDE_I_BPROJ_N_2000_bottleneck_16_firstc_6
DiscreteSBM_SigmaVPSDE_I_BPROJ_N_4000_bottleneck_16_firstc_6
DiscreteSBM_SigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_10
DiscreteSBM_SigmaVPSDE_I_BPROJ_N_2000_bottleneck_16_firstc_10
DiscreteSBM_SigmaVPSDE_I_BPROJ_N_4000_bottleneck_16_firstc_10


In [19]:
str_keys

'DiscreteSBM_SigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_6 DiscreteSBM_SigmaVPSDE_I_BPROJ_N_2000_bottleneck_32_firstc_6 DiscreteSBM_SigmaVPSDE_I_BPROJ_N_4000_bottleneck_32_firstc_6 DiscreteSBM_SigmaVPSDE_I_BPROJ_N_1000_bottleneck_32_firstc_10 DiscreteSBM_SigmaVPSDE_I_BPROJ_N_2000_bottleneck_32_firstc_10 DiscreteSBM_SigmaVPSDE_I_BPROJ_N_4000_bottleneck_32_firstc_10 DiscreteSBM_SigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_6 DiscreteSBM_SigmaVPSDE_I_BPROJ_N_2000_bottleneck_16_firstc_6 DiscreteSBM_SigmaVPSDE_I_BPROJ_N_4000_bottleneck_16_firstc_6 DiscreteSBM_SigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_10 DiscreteSBM_SigmaVPSDE_I_BPROJ_N_2000_bottleneck_16_firstc_10 DiscreteSBM_SigmaVPSDE_I_BPROJ_N_4000_bottleneck_16_firstc_10 '

In [20]:
with open(os.path.join(CONFIG_FOLDER,'exp9','config.json'), 'w') as f:
    json.dump(all_configs, f, indent=4)